
Cython test
===

In this little project we test the performance benefit from Cython over
Python by implementing projected gradient descent to solve
non-negative least-squares problems

$
\min_{x >= 0} ||Ax - b||^2.
$

This is a suitable test case since it has more complexity than e.g.
simple Fibonacci, but it is straightforward to compare.



As a benchmark, we will use the Lawson-Hanson method implemented in Scipy. We will test it on a moderately sized problem.

In [1]:

import numpy as np
# Import Lawson-Hanson method which we will use as a benchmark.
from scipy.optimize import nnls
from time import time


m = 5000
n = 3000

# Create random A and b.
A = np.random.randn(m, n)
b = np.random.randn(m)

In [2]:
# For benchmark, solve with Lawson-Hanson method.
t0 = time()
x_lh, rnorm_lh = nnls(A, b)
t_lh = time() - t0
print(f"Residual norm: {rnorm_lh}.")
print(f"Lawson-Hanson method took {t_lh} seconds.")

Residual norm: 60.10621032582238.
Lawson-Hanson method took 35.45346426963806 seconds.


So, this took quite some time. Next, we have implemented the *accelerated projected gradient descent* in pure Python. Let's see how it compares.



In [3]:
from src.apgd import accelerated_pgd

# Have to provide initial guess.
x0 = np.zeros(n)
num_iter = 1000

t0 = time()
x_pgd, rnorm_pgd = accelerated_pgd(a=A, b=b, x0=x0, num_iter=num_iter)
t_pgd = time() - t0
print(f"Residual norm: {rnorm_pgd}.")
print(f"Accelerated projected gradient descent took {t_pgd} seconds.")

Residual norm: 60.10621043028574.
Accelerated projected gradient descent took 2.4835667610168457 seconds.


We see that this method yields comparable results with a 10x-speedup.

Finally, how about our Cython implementation of the accelerated PGD method?


In [4]:
from apgd_cython import accelerated_pgd_cython

t0 = time()
x_cython = accelerated_pgd_cython(a=A, b=b, x0=x0, num_iter=num_iter)
t_cython = time() - t0
print(f"The Cython implementation took {t_cython} seconds.")

The Cython implementation took 2.3318402767181396 seconds.


Conclusion
---

In our particular case, Cython did not yield a speedup. The reason behind this is probably that most of the computations in `accelerated_pgd` are Numpy-operations
that cannot be further optimized using Cython.